In [ ]:
"""
Created on Mon Mar 04 17:32 2024

Apply weights to long historical period

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import random
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/ElmerIce_'+str(geoyear)+'/'



In [ ]:
geoyear = 2100

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'ElmerIce_4km_'+str(geoyear)+'isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]
file_isf = file_isf_nonnan.sel(Nisf=sorted_isf_rignot)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)

In [ ]:
rignot_isf = sorted_isf_rignot

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

CALVING

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')
calving_flux = davison_file['calving_obs']

ABUMIP GL FLUX

In [ ]:
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m_ElmerIcegeometries.nc')

In [ ]:
GL_flux_ABUMIP = GL_flux.sel(geoyear=geoyear)

SMB

In [ ]:
def start_end_year(mod, scen):
    if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
        ens_run = 'r1i1p1f2'
        to2300 = False
    elif mod in ['GISS-E2-1-H']:
        ens_run = 'r1i1p1f2'
        to2300 = True
    elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
        ens_run = 'r1i1p1f1'
        to2300 = True
    elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
        ens_run = 'r1i1p1f1'
        to2300 = False
    elif mod == 'UKESM1-0-LL':
        ens_run = 'r4i1p1f2'
        to2300 = True     
    elif mod == 'CESM2':
        ens_run = 'r11i1p1f1'
        to2300 = False        


    if scenario == 'historical':
        yystart = 1850 #1850
        yyend = 2014
    elif scenario == 'ssp245':
        yystart = 2015
        yyend = 2100  
    else:
        if to2300:
            yystart = 2015
            yyend = 2299
        else:
            yystart = 2015
            yyend = 2100  
    
    return ens_run, yystart, yyend

In [ ]:
melt_atmo_list2 = []
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out2/' #/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
#for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
#            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
#           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']:
    
melt_atmo3_list = []
for scenario in ['historical','ssp126','ssp245','ssp585']: #

    melt_atmo_list = []
    for mod in ['CESM2','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']: #'CNRM-CM6-1',
        
        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)
        
        if (scenario == 'historical') and (yystart == 1850):
            
            melt_atmo_1850 = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1850-1979.nc').sel(Nisf=rignot_isf)
            
            if (os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc')):
                melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                melt_atmo['time'] = melt_atmo['time'].dt.year
            else:
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                
            melt_atmo_list.append(xr.concat([melt_atmo_1850['SMB'],melt_atmo['SMB']], dim='time').assign_coords({'model': mod}))

        else:
            
            if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc'):
                melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo['time'] = melt_atmo['time'].dt.year
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
            else:
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))


    for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                'CanESM5','GFDL-CM4','GFDL-ESM4', 
               'MRI-ESM2-0']: #'GISS-E2-1-H',

        _, yystart, yyend = start_end_year(mod, scenario)
        #print(mod,yystart,yyend)
        
        if mod == 'GFDL-CM4' and scenario == 'ssp126':
            melt_atmo = melt_atmo * np.nan
        
        else:
            
            if (scenario == 'historical') and (yystart == 1850):
                    
                melt_atmo_1850 = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1850-1979.nc').sel(Nisf=rignot_isf)
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(xr.concat([melt_atmo_1850['SMB'],melt_atmo['SMB']], dim='time').assign_coords({'model': mod}))
            
            else:
            
                melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-'+str(yystart)+'-'+str(yyend)+'.nc').sel(Nisf=rignot_isf)
                melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))

    melt_atmo_scenario = xr.concat(melt_atmo_list, dim='model')
    melt_atmo3_list.append(melt_atmo_scenario.assign_coords({'scenario': scenario}))
melt_atmo_all0 = xr.concat(melt_atmo3_list, dim='scenario').sel(Nisf=rignot_isf)
#melt_atmo_all = xr.concat(melt_atmo_list2, dim='model')


In [ ]:
concat_melt_atmo_list = []

for scen in ['ssp126','ssp245','ssp585']:
    concat_melt_atmo = xr.concat([melt_atmo_all0.sel(time=range(1850,2015),scenario='historical').drop('scenario'), melt_atmo_all0.sel(time=range(2015,2300),scenario=scen).drop('scenario')], dim='time')
    concat_melt_atmo_list.append(concat_melt_atmo.assign_coords({'scenario': scen}))
melt_atmo_all = xr.concat(concat_melt_atmo_list, dim='scenario')

##### OCEAN

In [ ]:
Gt_list_models = []
box1_list_models = []

for mod in tqdm(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']): #'GISS-E2-1-H',
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP_ElmerIcegeo'+str(geoyear)+'.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP_ElmerIcegeo'+str(geoyear)+'.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod_anoISMIP = xr.concat(Gt_list_models, dim='model')
box1_all_mod_anoISMIP = xr.concat(box1_list_models, dim='model')


    
    

In [ ]:
Gt_list_models = []
box1_list_models = []

for mod in tqdm(['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']): #'GISS-E2-1-H',
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoNEMO_ElmerIcegeo'+str(geoyear)+'.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['ssp585']:
            if mod == 'GFDL-CM4' and scenario == 'ssp126':
                melt_1D = melt_1D * np.nan 
            else:
                melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoNEMO_ElmerIcegeo'+str(geoyear)+'.nc')
                melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod_anoNEMO = xr.concat(Gt_list_models, dim='model')
box1_all_mod_anoNEMO = xr.concat(box1_list_models, dim='model')


    
    

In [ ]:
ano_choice_file = xr.open_dataset(inputpath_weights + 'ano_choice_NEMOorISMIP_withoutGISS.nc')


bmb_combined_list = []
for kisf in tqdm(file_isf.Nisf):
    if ano_choice_file['ano_choice'].sel(Nisf=kisf).values == 'NEMO':
        bmb_combined_list.append(Gt_all_mod_anoNEMO.sel(Nisf=kisf))
    elif ano_choice_file['ano_choice'].sel(Nisf=kisf).values == 'ISMIP':
        bmb_combined_list.append(Gt_all_mod_anoISMIP.sel(Nisf=kisf))
bmb_combined0 = xr.concat(bmb_combined_list, dim='Nisf')

concat_bmb_list = []

bmb_combined = bmb_combined0.sel(time=range(geoyear,2300),scenario=scen)

In [ ]:
bmb_alldim, GL_flux_ABUMIP_alldim  = xr.broadcast(bmb_combined.sel(time=range(geoyear,2300)), GL_flux_ABUMIP['flux_Gt_ABUMIP'])
_, smb_alldim  = xr.broadcast(bmb_alldim.sel(time=range(geoyear,2300)), melt_atmo_all.sel(time=range(geoyear,2300)))
_, calv_alldim = xr.broadcast(bmb_alldim.sel(time=range(geoyear,2300)), davison_file['calving_obs'])

MASS BALANCE

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')

In [ ]:
mass_balance_simu = bmb_combined - melt_atmo_all + davison_file['calving_obs'] - GL_flux_ABUMIP['flux_Gt_ABUMIP']

In [ ]:
mass_balance_simu_2300 = mass_balance_simu.sel(model=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                        'CESM2-WACCM', 'IPSL-CM6A-LR',
                                                        'MRI-ESM2-0','UKESM1-0-LL']) #'GISS-E2-1-H',


WRITE INTO NETCDF FILE FOR FURTHER ANALYSIS IN OTHER NOTEBOOK

In [ ]:
GL_flux_ABUMIP_alldim

In [ ]:
file_viability_info = xr.Dataset()
file_viability_info['BMB'] = bmb_alldim
file_viability_info['SMB'] = smb_alldim
file_viability_info['ABUMIP'] = GL_flux_ABUMIP_alldim
file_viability_info['CALVING'] = calv_alldim
file_viability_info['MASS_BALANCE'] = mass_balance_simu
file_viability_info.to_netcdf(inputpath_weights + 'all_fluxes_br_withoutGISS_ElmerIcegeo'+str(geoyear)+'.nc')

In [ ]:
#bay_weights = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison.nc')['bay_weights']
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
ensmean_weighted = uf.weighted_mean(mass_balance_simu, ['model','param','m'], bay_weights * sens_weights.sel(model=mass_balance_simu.model))
ensmean_weighted_2300 = uf.weighted_mean(mass_balance_simu_2300, ['model','param','m'], bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model))

#ensmean_nonweighted = mass_balance_simu.mean(['model','param'])

In [ ]:
da_weighted = mass_balance_simu.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))
da_weighted_2300 = mass_balance_simu_2300.weighted(bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model))

In [ ]:
#ensstat_weighted = da_weighted.quantile([0.05,0.5,0.95],dim=['model','param'])
ensstat_weighted = da_weighted.quantile([0.05,0.1,0.33,0.66,0.5,0.9,0.95],dim=['model','param','m'])
ensstat_weighted_2300 = da_weighted_2300.quantile([0.05,0.1,0.33,0.66,0.5,0.9,0.95],dim=['model','param','m'])


VIABILITY LIMIT

In [ ]:
limit_50 = ensstat_weighted.sel(quantile = 0.5) > 0
limit_66 = ensstat_weighted.sel(quantile = 0.33) > 0
limit_90 = ensstat_weighted.sel(quantile = 0.10) > 0

In [ ]:
limit_50_2300 = ensstat_weighted_2300.sel(quantile = 0.5) > 0
limit_66_2300 = ensstat_weighted_2300.sel(quantile = 0.33) > 0
limit_90_2300 = ensstat_weighted_2300.sel(quantile = 0.10) > 0

In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

ll = 0
label_list = []

for scen in ['ssp126','ssp245','ssp585']: 
    
    if scen == 'ssp126':
        ccol = 'gold'
        shift=+0.3
    elif scen == 'ssp245':
        ccol = 'darkorange'
        shift=0
    elif scen == 'ssp585':
        ccol = 'red'
        shift=-0.3
    #elif scen == 'historical':
    #    ccol = 'grey'
    #    shift=0        
    

    yval = xr.DataArray(data=np.array(range(len(sorted_isf_rignot))), dims=['Nisf']).assign_coords({'Nisf': sorted_isf_rignot[::-1]})
    _ , yval_br = xr.broadcast(limit_66.time,yval)
    
    ll2=0
    #for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    for k,kisf in enumerate(yval.Nisf):
        time_nonvia50 = limit_50.time.where(limit_50.sel(Nisf=kisf,scenario=scen))
        time_nonvia66 = limit_66.time.where(limit_66.sel(Nisf=kisf,scenario=scen))
        time_nonvia90 = limit_90.time.where(limit_90.sel(Nisf=kisf,scenario=scen))
        
        lim50_yes = limit_50.sel(Nisf=kisf,scenario=scen).astype(int)
        lim66_yes = limit_66.sel(Nisf=kisf,scenario=scen).astype(int)
        lim90_yes = limit_90.sel(Nisf=kisf,scenario=scen).astype(int)

        plt.scatter(time_nonvia50,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim50_yes*0.33,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim66_yes*0.4,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia90,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim90_yes*1,s=10,edgecolors='None',color=ccol)
        
        
        if ll == 0:
            label_list.append(file_isf['isf_name'].sel(Nisf=kisf).values)
            
            if kisf in [69,59,48,50,22,40,41]:
                ll2 = ll2+1
                plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift*np.nan,alpha=lim66_yes*0.3,s=10,edgecolors='None',color=ccol)
                plt.axhline(yval_br.sel(Nisf=kisf).max().values+ll2,linestyle='--')
                label_list.append(' ')
                
        
        else:
            if kisf in [69,59,48,50,22,40,41]:                
                ll2 = ll2+1
        
    ll = ll+1
    
ax.set_yticks(np.arange(len(label_list)), labels=label_list)    
#ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2100)
#ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'scatterbarplots_allscen_until2100.png')


UNTIL 2300

In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

ll = 0
label_list = []

for scen in ['ssp126','ssp585']: #'historical',
    
    if scen == 'ssp126':
        ccol = 'gold'
        shift=+0.2
    elif scen == 'ssp245':
        ccol = 'darkorange'
        shift=0
    elif scen == 'ssp585':
        ccol = 'red'
        shift=-0.2
    elif scen == 'historical':
        ccol = 'grey'
        shift=0        
    

    yval = xr.DataArray(data=np.array(range(len(sorted_isf_rignot))), dims=['Nisf']).assign_coords({'Nisf': sorted_isf_rignot[::-1]})
    _ , yval_br = xr.broadcast(limit_66.time,yval)
    
    ll2=0
    #for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    for k,kisf in enumerate(yval.Nisf):
        time_nonvia50 = limit_50_2300.time.where(limit_50_2300.sel(Nisf=kisf,scenario=scen))
        time_nonvia66 = limit_66_2300.time.where(limit_66_2300.sel(Nisf=kisf,scenario=scen))
        time_nonvia90 = limit_90_2300.time.where(limit_90_2300.sel(Nisf=kisf,scenario=scen))

        lim50_yes = limit_50_2300.sel(Nisf=kisf,scenario=scen).astype(int)
        lim66_yes = limit_66_2300.sel(Nisf=kisf,scenario=scen).astype(int)
        lim90_yes = limit_90_2300.sel(Nisf=kisf,scenario=scen).astype(int)

        plt.scatter(time_nonvia50,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim50_yes*0.1,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim66_yes*0.2,s=10,edgecolors='None',color=ccol)
        plt.scatter(time_nonvia90,yval_br.sel(Nisf=kisf)+shift+ll2,alpha=lim90_yes*1,s=10,edgecolors='None',color=ccol)
        
        
        if ll == 0:
            label_list.append(file_isf['isf_name'].sel(Nisf=kisf).values)
            
            if kisf in [69,59,48,50,22,40,41]:
                ll2 = ll2+1
                plt.scatter(time_nonvia66,yval_br.sel(Nisf=kisf)+shift*np.nan,alpha=lim66_yes*0.3,s=10,edgecolors='None',color=ccol)
                plt.axhline(yval_br.sel(Nisf=kisf).max().values+ll2,linestyle='--')
                label_list.append(' ')
                
        
        else:
            if kisf in [69,59,48,50,22,40,41]:                
                ll2 = ll2+1
        
    ll = ll+1
    
ax.set_yticks(np.arange(len(label_list)), labels=label_list)    
#ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2300)
#ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'scatterbarplots_allscen_until2300.png')


PROBABILITY CURVES

In [ ]:
all_weights = bay_weights * sens_weights.sel(model=mass_balance_simu.model)
non_viable_mask = mass_balance_simu > 0
non_viable_sum = all_weights.where(non_viable_mask).sum(['model','param','m'])
all_weights_sum = all_weights.sum(['model','param','m'])
non_viable_normed = non_viable_sum/all_weights_sum

Define viability limit as a date

In [ ]:
viability_50_diff = limit_50.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit50_max = viability_50_diff.time.where(viability_50_diff == 0).max('time') + 1
limit50_max = limit50_max.where(limit50_max<2100)
limit50_max = limit50_max.where(~(np.isnan(limit50_max) & limit_50.sel(time=1850)), 1850)

viability_66_diff = limit_66.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit66_max = viability_66_diff.time.where(viability_66_diff == 0).max('time') + 1
limit66_max = limit66_max.where(limit66_max<2100)
limit66_max = limit66_max.where(~(np.isnan(limit66_max) & limit_66.sel(time=1850)), 1850)

viability_90_diff = limit_90.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit90_max = viability_90_diff.time.where(viability_90_diff == 0).max('time') + 1
limit90_max = limit90_max.where(limit90_max<2100)
limit90_max = limit90_max.where(~(np.isnan(limit90_max) & limit_90.sel(time=1850)), 1850)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)

    ax[i].plot(non_viable_normed.time.sel(time=range(1850,2015)), non_viable_normed.sel(Nisf=kisf,scenario='ssp126',time=range(1850,2015)), color='grey')    
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp126',time=range(2015,2100)), color='gold')
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp245',time=range(2015,2100)), color='darkorange')
    ax[i].plot(non_viable_normed.time.sel(time=range(2015,2100)), non_viable_normed.sel(Nisf=kisf,scenario='ssp585',time=range(2015,2100)), color='red')
    ax[i].axhline(y=0.50, color='deepskyblue', alpha=0.9, linestyle='--')
    ax[i].axhline(y=0.66, color='deepskyblue', alpha=0.5, linestyle='--')
    ax[i].axhline(y=0.90, color='deepskyblue', alpha=0.2, linestyle='--')
    ax[i].axvline(x=limit50_max.sel(Nisf=kisf, scenario='ssp585'), color='black', linestyle='--')

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' '+str(kisf))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    ax[i].set_xlim(1840,2099)
    ax[i].set_ylim(0,1)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'prob_distrib_allscen_until2100.pdf')


In [ ]:
all_weights_2300 = bay_weights_2300 * sens_weights.sel(model=mass_balance_simu_2300.model)
non_viable_mask_2300 = mass_balance_simu_2300 > 0
non_viable_sum_2300 = all_weights_2300.where(non_viable_mask_2300).sum(['model','param','m'])
all_weights_sum_2300 = all_weights_2300.sum(['model','param','m'])
non_viable_normed_2300 = non_viable_sum_2300/all_weights_sum_2300

Define viability limit as a date

In [ ]:
viability_50_diff = limit_50_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit50_max_2300 = viability_50_diff.time.where(viability_50_diff == 0).max('time') + 1
limit50_max_2300 = limit50_max_2300.where(limit50_max_2300<2299)
limit50_max_2300 = limit50_max_2300.where(~(np.isnan(limit50_max_2300) & limit_50_2300.sel(time=1850)), 1850)

viability_66_diff = limit_66_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit66_max_2300 = viability_66_diff.time.where(viability_66_diff == 0).max('time') + 1
limit66_max_2300 = limit66_max_2300.where(limit66_max_2300<2299)
limit66_max_2300 = limit66_max_2300.where(~(np.isnan(limit66_max_2300) & limit_66_2300.sel(time=1850)), 1850)

viability_90_diff = limit_90_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit90_max_2300 = viability_90_diff.time.where(viability_90_diff == 0).max('time') + 1
limit90_max_2300 = limit90_max_2300.where(limit90_max_2300<2299)
limit90_max_2300 = limit90_max_2300.where(~(np.isnan(limit90_max_2300) & limit_90_2300.sel(time=1850)), 1850)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(1850,2015)), non_viable_normed_2300.sel(Nisf=kisf,scenario='ssp126',time=range(1850,2015)), color='grey')    
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(2015,2300)), non_viable_normed_2300.sel(Nisf=kisf,scenario='ssp126',time=range(2015,2300)), color='gold')
    ax[i].plot(non_viable_normed_2300.time.sel(time=range(2015,2300)), non_viable_normed_2300.sel(Nisf=kisf,scenario='ssp585',time=range(2015,2300)), color='red')
    ax[i].axhline(y=0.50, color='deepskyblue', alpha=0.9, linestyle='--')
    ax[i].axhline(y=0.66, color='deepskyblue', alpha=0.5, linestyle='--')
    ax[i].axhline(y=0.90, color='deepskyblue', alpha=0.2, linestyle='--')
    ax[i].axvline(x=limit66_max_2300.sel(Nisf=kisf, scenario='ssp585'), color='black', linestyle='--')

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' '+str(kisf))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    ax[i].set_xlim(1840,2300)
    ax[i].set_ylim(0,1)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()

f.savefig(plot_path + 'prob_distrib_allscen_until2300.pdf')


In [ ]:
viability_limit_dates = xr.Dataset()
viability_limit_dates['viability_limit_2100'] = xr.concat([limit50_max.assign_coords({'quant': 50}),
                                                          limit66_max.assign_coords({'quant': 66}),
                                                          limit90_max.assign_coords({'quant': 90})], dim='quant')
viability_limit_dates['viability_limit_2300'] = xr.concat([limit50_max_2300.assign_coords({'quant': 50}),
                                                          limit66_max_2300.assign_coords({'quant': 66}),
                                                          limit90_max_2300.assign_coords({'quant': 90})], dim='quant')
viability_limit_dates.to_netcdf(inputpath_weights + 'viability_limits.nc')

ATTRIBUTING CHANGES TO SMB AND BMB

Compute viability limit for each ensemble member for 2100

In [ ]:
non_viable_ensmembers = mass_balance_simu > 0

In [ ]:
non_via_diff = non_viable_ensmembers.sel(time=range(2100,1849,-1)).cumsum('time').diff('time')
limit_via_max = non_via_diff.time.where(non_via_diff == 0).max('time') + 1
limit_via_max = limit_via_max.where(limit_via_max<2100)
limit_via_max = limit_via_max.where(~(np.isnan(limit_via_max) & non_viable_ensmembers.sel(time=1850)), 1850)

In [ ]:
stacked_limit_via_max = limit_via_max.stack(stack_dim = ['model','param','m']) #.sel(scenario='ssp585').sel(Nisf=66)
stacked_mass_balance_simu = mass_balance_simu.stack(stack_dim = ['model','param','m'])

Double-check that it is setting the limit at the right place

In [ ]:
i = random.randint(1, len(stacked_limit_via_max.stack_dim))
sd = stacked_limit_via_max.stack_dim.isel(stack_dim=i)
kisf=24

plt.figure()
stacked_mass_balance_simu.sel(scenario='ssp585').sel(Nisf=kisf).sel(stack_dim=sd.values).plot()
non_viable_time = stacked_limit_via_max.sel(scenario='ssp585').sel(Nisf=kisf).sel(stack_dim=sd).values
if np.isfinite(non_viable_time):
    print(i)
    plt.axvline(non_viable_time , linestyle='--')
plt.axhline(0,color='k')

In [ ]:
GL_flux_diff = GL_flux_ag['flux_Gt_ABUMIP'] - GL_flux_ag['flux_Gt_ref']
GL_flux_ABUMIP = GL_flux_ag['flux_Gt_ABUMIP']
SMB = - 1*melt_atmo_all
BMB = bmb_combined

In [ ]:
non_viable_time = limit_via_max

In [ ]:
GL_flux_ABUMIP_br, BMB_br = xr.broadcast(GL_flux_ag['flux_Gt_ABUMIP'], BMB)
SMB_br, _ = xr.broadcast(SMB, BMB_br)
time_br, _ = xr.broadcast(BMB_br.time, BMB_br)

In [ ]:
nonvia_GLflux = GL_flux_ABUMIP_br.where(time_br == non_viable_time, drop=True).max('time')
nonvia_SMB = SMB_br.where(time_br == non_viable_time, drop=True).max('time')
nonvia_BMB = BMB_br.where(time_br == non_viable_time, drop=True).max('time')

In [ ]:
nonvia_GLflux_stacked = nonvia_GLflux.stack(stack_dim=['m','param','model'])
nonvia_SMB_stacked = nonvia_SMB.stack(stack_dim=['m','param','model'])
nonvia_BMB_stacked = nonvia_BMB.stack(stack_dim=['m','param','model'])

In [ ]:
all_weights = bay_weights * sens_weights.sel(model=mass_balance_simu.model)

In [ ]:
nonvia_BMB_weighted = nonvia_BMB.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))
nonvia_SMB_weighted = nonvia_SMB.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))
nonvia_GLflux_weighted = nonvia_GLflux.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model))


In [ ]:
BMB_quantiles = nonvia_BMB_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])
SMB_quantiles = nonvia_SMB_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])
GLflux_quantiles = nonvia_GLflux_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])

In [ ]:
width_BMB_inner = BMB_quantiles.sel(quantile=0.66) - BMB_quantiles.sel(quantile=0.33)
width_BMB_outer = BMB_quantiles.sel(quantile=0.9) - BMB_quantiles.sel(quantile=0.1)
width_BMB_inner = width_BMB_inner.where(np.isfinite(width_BMB_inner), 0)
width_BMB_outer = width_BMB_outer.where(np.isfinite(width_BMB_outer), 0)

In [ ]:
width_SMB_inner = SMB_quantiles.sel(quantile=0.66) - SMB_quantiles.sel(quantile=0.33)
width_SMB_outer = SMB_quantiles.sel(quantile=0.9) - SMB_quantiles.sel(quantile=0.1)
width_SMB_inner = width_SMB_inner.where(np.isfinite(width_SMB_inner), 0)
width_SMB_outer = width_SMB_outer.where(np.isfinite(width_SMB_outer), 0)

In [ ]:

for mmod in SMB.model:
    plt.figure()
    #SMB.sum('Nisf').sel(model=mmod).sel(scenario='historical').plot(color='grey')
    #SMB.sum('Nisf').sel(model=mmod).sel(scenario='ssp126').plot(color='gold')
    #SMB.sum('Nisf').sel(model=mmod).sel(scenario='ssp245').plot(color='orange')
    SMB.sum('Nisf').sel(model=mmod).sel(scenario='ssp585').plot(color='red')
    plt.title(mmod.values)

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    #width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        if kisf in width_BMB_inner.Nisf.values.tolist():
            ax.barh(k+0.35, width=width_BMB_inner.sel(Nisf=kisf,scenario=scen), left=BMB_quantiles.sel(quantile=0.33).sel(Nisf=kisf,scenario=scen), height=0.35, color='blue',alpha=0.8)
            ax.barh(k+0.35, width=width_BMB_outer.sel(Nisf=kisf,scenario=scen), left=BMB_quantiles.sel(quantile=0.1).sel(Nisf=kisf,scenario=scen), height=0.35, color='blue',alpha=0.2)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            ax.barh(k-0.35, width=width_SMB_inner.sel(Nisf=kisf,scenario=scen), left=SMB_quantiles.sel(quantile=0.33).sel(Nisf=kisf,scenario=scen), height=0.35, color='orange',alpha=0.8)
            ax.barh(k-0.35, width=width_SMB_outer.sel(Nisf=kisf,scenario=scen), left=SMB_quantiles.sel(quantile=0.1).sel(Nisf=kisf,scenario=scen), height=0.35, color='orange',alpha=0.2)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    #ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()

In [ ]:
##### CHECK EVERY TERM FOR ONE ICE SHELF

In [ ]:
kisf = 41
scen = 'ssp585'

plt.figure()
for mod in mass_balance_simu.model:
    for mparam in mass_balance_simu.param:
        bmb_combined.sel(model=mod,param=mparam,Nisf=kisf,scenario=scen,time=range(1850,2101)).plot(color='blue', alpha=0.2)
plt.title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' basal melt')


In [ ]:
kisf = 41
scen = 'ssp585'

plt.figure()
for mod in mass_balance_simu.model:
    (-1*melt_atmo_all.sel(model=mod,Nisf=kisf,scenario=scen,time=range(1850,2101))).plot(color='orange', alpha=0.2)
plt.title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' SMB * -1')


In [ ]:
kisf = 41
scen = 'ssp585'

plt.figure()
plt.axhline(calv.sel(Nisf=kisf),color='olive')
plt.title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' calving')


In [ ]:
kisf = 41
scen = 'ssp585'

plt.figure()
for mod in mass_balance_simu.m:
    plt.axhline(GL_flux_ag['flux_Gt_ABUMIP'].sel(m=m,Nisf=kisf),color='black', alpha=0.2)
plt.title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' ABUMIP limit')


In [ ]:
##############

In [ ]:
nonvia_SMB = nonvia_SMB 
nonvia_BMB = nonvia_BMB
calv = davison_file['calving_obs']

In [ ]:
nonvia_SMB_mean = nonvia_SMB.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])
nonvia_BMB_mean = nonvia_BMB.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])
calv_mean = calv.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    #width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        
        if kisf in width_BMB_inner.Nisf.values.tolist():
            
            if nonvia_SMB_mean.sel(Nisf=kisf,scenario=scen) > 0:
                lleft = 0
            else:
                lleft = nonvia_SMB_mean.sel(Nisf=kisf,scenario=scen)
                
            ax.barh(k+0.2, width=GL_flux_ABUMIP.sel(Nisf=kisf).min(), left=0, height=0.4, color='grey',alpha=1.0)
            ax.barh(k+0.2, width=GL_flux_ABUMIP.sel(Nisf=kisf).median(), left=GL_flux_ABUMIP.sel(Nisf=kisf).min(), height=0.4, color='grey',alpha=0.6)
            ax.barh(k+0.2, width=GL_flux_ABUMIP.sel(Nisf=kisf).max(), left=GL_flux_ABUMIP.sel(Nisf=kisf).min() + GL_flux_ABUMIP.sel(Nisf=kisf).median(), height=0.4, color='grey',alpha=0.3)
            
            ax.barh(k-0.2, width=np.abs(nonvia_SMB_mean.sel(Nisf=kisf,scenario=scen)), left=lleft, height=0.4, color='orange')
            ax.barh(k-0.2, width=nonvia_BMB_mean.sel(Nisf=kisf,scenario=scen), left=lleft+np.abs(nonvia_SMB_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='blue')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            ax.barh(k-0.2, width=calv.sel(Nisf=kisf),left=nonvia_BMB_mean.sel(Nisf=kisf,scenario=scen) + lleft+np.abs(nonvia_SMB_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='olive')
            
    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    #ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()

Look where ABUMIP is below Davison calving flux

In [ ]:
for kisf in sorted_isf_rignot:
    if GL_flux_ABUMIP.sel(Nisf=kisf).min() < calv.sel(Nisf=kisf):
        print(kisf, file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
nonvia_SMB_perc = nonvia_SMB / nonvia_GLflux * 100
nonvia_BMB_perc = nonvia_BMB / nonvia_GLflux * 100
calv_perc = davison_file['calving_obs'] / nonvia_GLflux * 100

In [ ]:
nonvia_SMB_perc_mean = nonvia_SMB_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])
nonvia_BMB_perc_mean = nonvia_BMB_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])
calv_perc_mean = calv_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).mean(['param','model','m'])

In [ ]:
nonvia_BMB_perc_mean.mean('Nisf')

In [ ]:
calv_perc_mean.mean('Nisf')

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    #width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        if kisf in width_BMB_inner.Nisf.values.tolist():
            #ax.barh(k+0.2, width=GLflux_quantiles.sel(quantile=0.5).sel(Nisf=kisf,scenario=scen), left=0, height=0.4, color='deepskyblue',alpha=0.4)
            if nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen) > 0:
                lleft = 0
            else:
                lleft = nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)
            ax.barh(k, width=np.abs(nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)), left=lleft, height=0.4, color='orange')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            ax.barh(k, width=nonvia_BMB_perc_mean.sel(Nisf=kisf,scenario=scen), left=lleft+np.abs(nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='blue')
            ax.barh(k, width=calv_perc_mean.sel(Nisf=kisf,scenario=scen), left=nonvia_BMB_perc_mean.sel(Nisf=kisf,scenario=scen) + lleft+np.abs(nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='olive')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            
            #if lleft == nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen):
            #    ax.barh(k, width=np.abs(nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)), left=nonvia_BMB_perc_mean.sel(Nisf=kisf,scenario=scen) + lleft + calv_perc_mean.sel(Nisf=kisf,scenario=scen), hatch=r"\\", facecolor='white', height=0.4, alpha=0.6)
                
            #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    #ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    plt.savefig(plot_path + 'barplots_percentage_of_ABUMIP_until2100_'+scen+'.pdf')

In [ ]:
nonvia_SMB_perc_median = nonvia_SMB_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).quantile(0.5, dim=['param','model','m'])
nonvia_BMB_perc_median = nonvia_BMB_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu.model)).quantile(0.5, dim=['param','model','m'])

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    #width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        if kisf in width_BMB_inner.Nisf.values.tolist():
            #ax.barh(k+0.2, width=GLflux_quantiles.sel(quantile=0.5).sel(Nisf=kisf,scenario=scen), left=0, height=0.4, color='deepskyblue',alpha=0.4)
            ax.barh(k, width=nonvia_BMB_perc_median.sel(Nisf=kisf,scenario=scen), left=0, height=0.4, color='blue')
            ax.barh(k, width=nonvia_SMB_perc_median.sel(Nisf=kisf,scenario=scen), left=nonvia_BMB_perc_median.sel(Nisf=kisf,scenario=scen), height=0.4, color='orange')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    #ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()

Compute viability limit for 2300

In [ ]:
non_viable_ensmembers_2300 = mass_balance_simu_2300 > 0

non_via_diff_2300 = non_viable_ensmembers_2300.sel(time=range(2299,1849,-1)).cumsum('time').diff('time')
limit_via_max_2300 = non_via_diff_2300.time.where(non_via_diff_2300 == 0).max('time') + 1
limit_via_max_2300 = limit_via_max_2300.where(limit_via_max_2300<2299)
limit_via_max_2300 = limit_via_max_2300.where(~(np.isnan(limit_via_max_2300) & non_viable_ensmembers_2300.sel(time=1850)), 1850)

In [ ]:
GL_flux_diff = GL_flux_ag['flux_Gt_ABUMIP'] - GL_flux_ag['flux_Gt_ref']
GL_flux_ABUMIP = GL_flux_ag['flux_Gt_ABUMIP']
SMB_2300 = - 1*melt_atmo_all.sel(model=mass_balance_simu_2300.model)
BMB_2300 = bmb_combined.sel(model=mass_balance_simu_2300.model)

In [ ]:
non_viable_time_2300 = limit_via_max_2300

In [ ]:
GL_flux_ABUMIP_br, BMB_2300_br = xr.broadcast(GL_flux_ag['flux_Gt_ABUMIP'], BMB_2300)
SMB_2300_br, _ = xr.broadcast(SMB_2300, BMB_2300_br)
time_2300_br, _ = xr.broadcast(BMB_2300_br.time, BMB_2300_br)

In [ ]:
nonvia_GLflux = GL_flux_ABUMIP_br.where(time_br == non_viable_time_2300, drop=True).max('time')
nonvia_SMB_2300 = SMB_2300_br.where(time_br == non_viable_time_2300, drop=True).max('time')
nonvia_BMB_2300 = BMB_2300_br.where(time_br == non_viable_time_2300, drop=True).max('time')

In [ ]:
all_weights_2300 = bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model)

In [ ]:
nonvia_BMB_2300_weighted = nonvia_BMB_2300.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model))
nonvia_SMB_2300_weighted = nonvia_SMB_2300.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model))
nonvia_GLflux_2300_weighted = nonvia_GLflux.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model))

In [ ]:
BMB_2300_quantiles = nonvia_BMB_2300_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])
SMB_2300_quantiles = nonvia_SMB_2300_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])
GLflux_2300_quantiles = nonvia_GLflux_2300_weighted.quantile([0.1,0.33,0.5,0.66,0.9],dim=['model','param','m'])

In [ ]:
nonvia_SMB_2300_perc = nonvia_SMB_2300 / nonvia_GLflux * 100
nonvia_BMB_2300_perc = nonvia_BMB_2300 / nonvia_GLflux * 100
calv_perc = davison_file['calving_obs'] / nonvia_GLflux * 100

In [ ]:
nonvia_SMB_2300_perc_mean = nonvia_SMB_2300_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model)).mean(['param','model','m'])
nonvia_BMB_2300_perc_mean = nonvia_BMB_2300_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model)).mean(['param','model','m'])
calv_2300_perc_mean = calv_perc.weighted(bay_weights * sens_weights.sel(model=mass_balance_simu_2300.model)).mean(['param','model','m'])

In [ ]:
calv_2300_perc_mean.mean('Nisf')

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    #width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        if kisf in mass_balance_simu_2300.Nisf.values.tolist():
            #ax.barh(k+0.2, width=GLflux_quantiles.sel(quantile=0.5).sel(Nisf=kisf,scenario=scen), left=0, height=0.4, color='deepskyblue',alpha=0.4)
            if nonvia_SMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen) > 0:
                lleft = 0
            else:
                lleft = nonvia_SMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen)
            ax.barh(k, width=np.abs(nonvia_SMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen)), left=lleft, height=0.4, color='orange')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            ax.barh(k, width=nonvia_BMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen), left=lleft+np.abs(nonvia_SMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='blue')
            ax.barh(k, width=calv_2300_perc_mean.sel(Nisf=kisf,scenario=scen), left=nonvia_BMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen) + lleft+np.abs(nonvia_SMB_2300_perc_mean.sel(Nisf=kisf,scenario=scen)), height=0.4, color='olive')#,yerr=d["{0}_per1_all_std".format(var)][4,:])
            
            #if lleft == nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen):
            #    ax.barh(k, width=np.abs(nonvia_SMB_perc_mean.sel(Nisf=kisf,scenario=scen)), left=nonvia_BMB_perc_mean.sel(Nisf=kisf,scenario=scen) + lleft + calv_perc_mean.sel(Nisf=kisf,scenario=scen), hatch=r"\\", facecolor='white', height=0.4, alpha=0.6)
                
            #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    #ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    plt.savefig(plot_path + 'barplots_percentage_of_ABUMIP_until2300_'+scen+'.pdf')

Then do a weighted mean

In [ ]:
BMB_simu_list = []
for kisf in tqdm(file_isf.Nisf):
    if ano_and_weights['ano_choice'].sel(Nisf=kisf).values == 'NEMO':
        BMB_simu_list.append(Gt_all_mod_anoNEMO.sel(Nisf=kisf))
    elif ano_and_weights['ano_choice'].sel(Nisf=kisf).values == 'ISMIP':
        BMB_simu_list.append(Gt_all_mod_anoISMIP.sel(Nisf=kisf))
BMB_simu = xr.concat(BMB_simu_list, dim='Nisf')

In [ ]:
BMB_perc = (BMB_simu / mass_balance_simu).where(non_viable_mask)
SMB_perc = (melt_atmo_all / mass_balance_simu).where(non_viable_mask)

Need to think more about this!!!!!! It is tricky

VIABILITY LIMIT (this was dividing the m's, not needed anymore)

In [ ]:
limit_dates = xr.Dataset()

# 66 % above lowest ABUMIP limit
limit_dates['lowest_66'] = ensstat_weighted.sel(quantile=0.66) > 0

# 90 % above lowest ABUMIP limit
limit_dates['lowest_90'] = ensstat_weighted.sel(quantile=0.90) > GL_flux_ABUMIP_inone.min('m')

# 66 % above highest ABUMIP limit
limit_dates['highest_66'] = ensstat_weighted.sel(quantile=0.66) > GL_flux_ABUMIP_inone.max('m')

# 90 % above highest ABUMIP limit
limit_dates['highest_90'] = ensstat_weighted.sel(quantile=0.90) > GL_flux_ABUMIP_inone.max('m')

In [ ]:
kisf=66
scen = 'ssp585'

limit_dates['highest_66'].sel(scenario='ssp585', Nisf=kisf).plot()

In [ ]:
for stat in ['lowest_66','highest_66','lowest_90','highest_90']:
    fyear = limit_dates[stat].cumsum('time')
    time_br, _ = xr.broadcast(limit_dates.time, limit_dates)
    limit_dates[stat+'_threshold'] = time_br.where(fyear == 1).max('time')
    limit_dates[stat+'_threshold'] = limit_dates[stat+'_threshold'].where(np.isfinite(limit_dates[stat+'_threshold']),2014)

In [ ]:
width = xr.Dataset()
#for scen in ['historical','ssp126','ssp245','ssp585']:
    #allmax = limit_dates['highest_66_threshold'].where(limit_dates['highest_90_threshold'] <= limit_dates['highest_66_threshold'], limit_dates['highest_90_threshold'])
    #allmin = limit_dates['lowest_66_threshold'].where(limit_dates['lowest_90_threshold'] >= limit_dates['lowest_66_threshold'], limit_dates['lowest_90_threshold'])
width['range66'] = limit_dates['highest_66_threshold'] - limit_dates['lowest_66_threshold']
width['range90'] = limit_dates['highest_90_threshold'] - limit_dates['lowest_90_threshold']
    #'historical',

In [ ]:
scen='ssp585'
kisf = 66

limit_dates_list = []
for stat in ['lowest_66','highest_66','lowest_90','highest_90']:
    print(stat)
    limit_dates_scen_list = []
    for scen in ['historical','ssp126','ssp245','ssp585']:
        if scen == 'historical':
            limit_dates[stat] = 1850 + (limit_dates[stat+'_threshold'].sel(scenario=scen) == False).sum('time')
        else:
            limit_dates[stat] = 2014 + (limit_dates[stat+'_threshold'].sel(scenario=scen) == False).sum('time')
        limit_dates_scen_list.append(limit_dates[stat].assign_coords({'scenario': scen}))
    limit_dates_scen = xr.concat(limit_dates_scen_list, dim='scenario')
    limit_dates_list.append(limit_dates_scen)
limit_dates_all = xr.merge(limit_dates_list)




In [ ]:
ensstat_weighted['quantile']

In [ ]:
width_plot['historical_66'].sel(Nisf=kisf)

In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.35, color=ccol,alpha=0.4)
        ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.35, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
        #ax.barh(k, width=2300-limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), left=limit_dates['highest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.4, color='darkgrey',hatch='/',alpha=0.9)
        
    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    ax.set_xlim(2014,2100)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    
    fig.savefig(plot_path + 'barplots_'+scen+'_until2100.png')


In [ ]:
for scen in ['ssp126','ssp245','ssp585']:
    
    if scen == 'ssp126':
        ccol = 'gold'
    elif scen == 'ssp245':
        ccol = 'darkorange'
    elif scen == 'ssp585':
        ccol = 'red'
    
    fig, ax = plt.subplots()
    fig1 = plt.gcf()
    fig.set_size_inches(8.25, 8.25*1.5)

    yval = np.array(range(len(sorted_isf_rignot)))
    width_plot = width.where(np.isfinite(width), 0)

    for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
        ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)
        ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

    ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
    #ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
    ax.set_xlim(2014,2300)
    ax.set_title(scen)

    sns.despine()
    plt.grid(linestyle='--')
    plt.tight_layout()
    
    
    fig.savefig(plot_path + 'barplots_'+scen+'_until2300.png')


In [ ]:
scen = 'historical'
ccol = 'darkgrey'


fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*1.5)

yval = np.array(range(len(sorted_isf_rignot)))
width_plot = width.where(np.isfinite(width), 0)

for k,kisf in enumerate(file_isf.Nisf.sel(Nisf=sorted_isf_rignot[::-1])):
    ax.barh(k, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)
    ax.barh(k, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.8, color=ccol,alpha=0.4)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

ax.set_yticks(yval, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot[::-1]).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(1850,2014)
ax.set_title(scen)

sns.despine()
plt.grid(linestyle='--')
plt.tight_layout()

fig.savefig(plot_path + 'barplots_'+scen+'.png')


In [ ]:
fig, ax = plt.subplots()
fig1 = plt.gcf()
fig.set_size_inches(8.25, 8.25*2)

yval = np.array(range(len(sorted_isf_rignot)))
width_plot = width.where(np.isfinite(width), 0)

for kisf in file_isf.Nisf.sel(Nisf=sorted_isf_rignot):
    scen = 'ssp126'
    ax.barh(kisf, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='gold',alpha=0.5)
    ax.barh(kisf, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='gold',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    scen = 'ssp245'
    ax.barh(kisf+0.25, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='darkorange',alpha=0.5)
    ax.barh(kisf+0.25, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='darkorange',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])
    scen = 'ssp585'
    ax.barh(kisf+0.5, width=width_plot['range66'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_66_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='red',alpha=0.5)
    ax.barh(kisf+0.5, width=width_plot['range90'].sel(Nisf=kisf,scenario=scen), left=limit_dates['lowest_90_threshold'].sel(Nisf=kisf,scenario=scen), height=0.2, color='red',alpha=0.5)#,yerr=d["{0}_per1_all_std".format(var)][4,:])

ax.set_yticks(yval+0.25, labels=file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
#ax.set_yticklabels(file_isf['isf_name'].sel(Nisf=sorted_isf_rignot).values)
ax.set_xlim(2014,2100)

sns.despine()
plt.grid()
